In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn import under_sampling, over_sampling
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report 
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:

data=pd.read_csv('player-classify-cleaned.csv')
# print (data.isnull().sum())a

In [3]:
# data.head()

In [4]:
data = data.iloc[: , 1:]

In [5]:

data['value']=data['value'].replace({'A':1,'B':2,'C':3,'D':4,'S':5})
y = data['value']
x = data.iloc[:, :-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=8)
# print (y.head())

In [6]:
# x_train.dtypes

In [7]:

#get correlations of each features in dataset
corr_df = data.corr(method ='kendall').sort_values(by=['value']).iloc[:-1,-1]
# print (corr_df)
# feature_nonan =  dict(corr_df)
# x = x[top_feature]

In [8]:
len(corr_df)

78

In [9]:
corr_df[-6:]

contract_duration                0.038875
value                            1.000000
preferred_foot                        NaN
international_reputation(1-5)         NaN
weak_foot(1-5)                        NaN
Wark Rate Detensivel                  NaN
Name: value, dtype: float64

In [10]:
corr_df.dropna(inplace=True)

In [11]:
corr_df = corr_df.iloc[0:-1]

In [12]:
# corr_df[0:]

In [13]:
best_feature = dict(corr_df[0:])
x= data[best_feature.keys()]
x.shape

(14136, 73)

In [14]:
best_feature

{'release_clause_euro': -0.4184731119619384,
 'overall_rating': -0.3140582848420411,
 'wage': -0.23610058877205609,
 'ball_control': -0.17499871667858477,
 'reactions': -0.17362283634349848,
 'potential': -0.152942059337219,
 'RM': -0.15222631177160756,
 'LM': -0.15203741502169243,
 'composure': -0.1504731270295203,
 'skill_moves(1-5)': -0.14046642490910696,
 'CAM': -0.13548940381644534,
 'LAM': -0.1354426517620435,
 'RAM': -0.1354272638635907,
 'LCM': -0.13533691277717233,
 'short_passing': -0.13524821433118125,
 'RCM': -0.1350642420645306,
 'CM': -0.13494538272997422,
 'dribbling': -0.12702740849454797,
 'LF': -0.12679410355179466,
 'CF': -0.1267794434405475,
 'RF': -0.12675428168895458,
 'RW': -0.12170241512428918,
 'LW': -0.12145455412532911,
 'shot_power': -0.11691322537928905,
 'positioning': -0.10886262886398228,
 'long_shots': -0.10844574609642708,
 'long_passing': -0.10387192593128858,
 'vision': -0.10104680976211143,
 'crossing': -0.10100215857618167,
 'heading_accuracy': -0.

In [15]:
 
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=8)

In [16]:
# y_train.value_counts()

In [17]:
# pip install -U imbalanced-learn

In [18]:
# check version number
# print(imblearn.__version__)

In [19]:

scaler=StandardScaler().fit(X_train)
# save the model to disk
filename = 'scaler.sav'
pickle.dump(scaler, open(filename, 'wb')) 
X_train=pd.DataFrame(scaler.transform(X_train))
X_test=pd.DataFrame(scaler.transform(X_test))

In [20]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=2)
X_train_res,y_train_res=sm.fit_resample(X_train,y_train)

In [21]:
# summarize the new class distribution
from collections import Counter
counter = Counter(y_train_res)
print(counter)

Counter({5: 2954, 3: 2954, 1: 2954, 4: 2954, 2: 2954})


In [22]:
model_svc=SVC()
model_svc.fit(X_train,y_train)
y_pred=model_svc.predict(X_test)

filename = 'model_svc.sav'
pickle.dump(model_svc, open(filename, 'wb')) 
print(accuracy_score(y_test,y_pred)*100)
# 0.7661498708010336
print(classification_report(y_test,y_pred))

87.44695898161244
              precision    recall  f1-score   support

           1       0.88      0.86      0.87       619
           2       0.77      0.83      0.80       448
           3       0.87      0.88      0.88       765
           4       0.94      0.91      0.92       650
           5       0.90      0.89      0.89       346

    accuracy                           0.87      2828
   macro avg       0.87      0.87      0.87      2828
weighted avg       0.88      0.87      0.88      2828



In [23]:
model_dt=DecisionTreeClassifier(class_weight='balanced',max_depth=5)
model_dt.fit(X_train_res,y_train_res)
y_pred=model_dt.predict(X_test)

filename = 'model_dt.sav'
pickle.dump(model_dt, open(filename, 'wb')) 
print(accuracy_score(y_test,y_pred))
# 0.7609819121447028
print(classification_report(y_test,y_pred))

0.8613861386138614
              precision    recall  f1-score   support

           1       0.87      0.85      0.86       619
           2       0.67      0.82      0.74       448
           3       0.90      0.81      0.86       765
           4       0.96      0.92      0.94       650
           5       0.88      0.92      0.90       346

    accuracy                           0.86      2828
   macro avg       0.86      0.87      0.86      2828
weighted avg       0.87      0.86      0.86      2828



In [24]:
model_rf=RandomForestClassifier(class_weight='balanced',random_state=1,max_depth=5,\
                                  n_estimators=10,max_features=1)
model_rf.fit(X_train_res,y_train_res)
ypred=model_rf.predict(X_test)

filename = 'model_rf.sav'
pickle.dump(model_rf, open(filename, 'wb')) 
print(accuracy_score(y_test,y_pred))
# 0.5930232558139535
print(classification_report(y_test,y_pred))

0.8613861386138614
              precision    recall  f1-score   support

           1       0.87      0.85      0.86       619
           2       0.67      0.82      0.74       448
           3       0.90      0.81      0.86       765
           4       0.96      0.92      0.94       650
           5       0.88      0.92      0.90       346

    accuracy                           0.86      2828
   macro avg       0.86      0.87      0.86      2828
weighted avg       0.87      0.86      0.86      2828



In [25]:
model_knn=KNeighborsClassifier(5)
model_knn.fit(X_train_res,y_train_res)
y_pred=model_knn.predict(X_test)

filename = 'model_knn.sav'
pickle.dump(model_knn, open(filename, 'wb')) 
print(accuracy_score(y_test,y_pred))
# 0.6821705426356589
print(classification_report(y_test,y_pred))

0.6941301272984441
              precision    recall  f1-score   support

           1       0.79      0.62      0.69       619
           2       0.45      0.74      0.56       448
           3       0.72      0.53      0.61       765
           4       0.86      0.81      0.83       650
           5       0.74      0.91      0.82       346

    accuracy                           0.69      2828
   macro avg       0.71      0.72      0.70      2828
weighted avg       0.73      0.69      0.70      2828



In [26]:
model_ada=AdaBoostClassifier(learning_rate=0.01)
model_ada.fit(X_train_res,y_train_res)
y_pred=model_ada.predict(X_test)

filename = 'model_ada.sav'
pickle.dump(model_ada, open(filename, 'wb')) 
print(accuracy_score(y_test,y_pred))
# 0.8126614987080103
print(classification_report(y_test,y_pred))

0.6138613861386139
              precision    recall  f1-score   support

           1       0.86      0.71      0.78       619
           2       0.55      0.85      0.67       448
           3       0.00      0.00      0.00       765
           4       0.49      0.94      0.64       650
           5       0.83      0.87      0.85       346

    accuracy                           0.61      2828
   macro avg       0.55      0.68      0.59      2828
weighted avg       0.49      0.61      0.53      2828



C:\Users\a\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\a\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\a\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:

ada_clf=AdaBoostClassifier()
params={
     'n_estimators':np.arange(10,300,10),
     'learning_rate':[0.01,0.05,0.1,1]
}
classes=Counter(y_train_res).keys()
print(classes)

dict_keys([5, 3, 1, 4, 2])


In [28]:

param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [1, 2]
             }


DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)

ABC = AdaBoostClassifier(base_estimator = DTC)

# run grid search
grid_search_ABC = GridSearchCV(ABC, param_grid=param_grid, scoring = 'f1_weighted',error_score='raise')

grid_search_ABC.fit(X_train_res,y_train_res)

print("The best parameters are:",grid_search_ABC.best_params_)

The best parameters are: {'base_estimator__criterion': 'gini', 'base_estimator__splitter': 'best', 'n_estimators': 2}


In [29]:
y_pred=grid_search_ABC.predict(X_test)

filename = 'grid_search_ABC.sav'
pickle.dump(grid_search_ABC, open(filename, 'wb')) 
print(accuracy_score(y_test,y_pred))
# 0.8036175710594315
print(classification_report(y_test,y_pred))

0.8550212164073551
              precision    recall  f1-score   support

           1       0.86      0.84      0.85       619
           2       0.75      0.78      0.77       448
           3       0.86      0.85      0.85       765
           4       0.91      0.91      0.91       650
           5       0.88      0.88      0.88       346

    accuracy                           0.86      2828
   macro avg       0.85      0.85      0.85      2828
weighted avg       0.86      0.86      0.86      2828

